<a href="https://colab.research.google.com/github/tkami22/scraping_test/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Googleドライブへのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


# アクセス可否の確認

In [ ]:
# import urllib.robotparser

# # robots.txtの読み取り
# robots_txt_url = 'https://www.nomurakougei.co.jp/robots.txt'
# rp = urllib.robotparser.RobotFileParser()
# rp.set_url(robots_txt_url)
# rp.read()

# # robots.txtの情報から調査したいURL、User-Agentでクロール可能かを調べる
# user_agent = '*'
# url = 'https://www.nomurakougei.co.jp/achievements/*'
# result = rp.can_fetch(user_agent, url)
# print(result)

True


# コード

## 事前設定

In [55]:
# 必要ライブラリのインポート
import urllib
from urllib.error import URLError
from requests.models import HTTPError
from bs4 import BeautifulSoup

# デバッグ用
import sys
import pprint

In [2]:
# soupを作成する関数
def MakeSoup(url, ua):
  req = urllib.request.Request(url, headers={'User-Agent': ua})
  try:
    html = urllib.request.urlopen(req)
  except HTTPError as e:
    print('HTTPError')
  except URLError as e:
    print('URLError')
  else:
    if not html.geturl() == url:
      print('Redirected')
    else:
      return BeautifulSoup(html, "html.parser")

In [3]:
# トップページURL
url_root = 'https://www.nomurakougei.co.jp'
# ユーザーエージェント
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'

## [すべての実績一覧](https://www.nomurakougei.co.jp/achievements/all)

In [4]:
# すべての実績一覧 ページURLを全て取得する関数
def Get_AchievementsAll_PageUrl_list(url_root):
  url_achievementsAll = url_root + '/achievements/all'
  url_achievementsPages_list = []
  i = 1
  while True:
    i_url_achievements = url_achievementsAll + f'?page={i}'
    i_soup_achievements = MakeSoup(i_url_achievements, ua)
    if not i_soup_achievements is None:
      url_achievementsPages_list.append(i_url_achievements)
      i += 1
    else:
      break
  return url_achievementsPages_list

In [5]:
# ページリンクを取得
url_achievementsPages_list = Get_AchievementsAll_PageUrl_list(url_root)
# 個別実績タイトル、リンクを全て取得
title_achievement_list = []
url_achievement_list = []
for page_num, url_achievementsPage in enumerate(url_achievementsPages_list):
  soup_achievementsPage = MakeSoup(url_achievementsPage, ua)
  ul_a_list = soup_achievementsPage.article.find('ul', {"class": "wall col-3"}).find_all("a")
  for i_ul_a in ul_a_list:
    i_ul_title = i_ul_a.get('title')
    i_ul_href = i_ul_a.get('href')
    title_achievement_list.append(i_ul_title)
    url_achievement_list.append(url_root + i_ul_href)

Redirected


## 個別実績

In [69]:
# # 個別実績タイトル、リンクのリスト
# title_achievement_list = []
# url_achievement_list = []

# 個別実績ページ
for i, url_achievement in enumerate(url_achievement_list):
  soup_achievement = MakeSoup(url_achievement, ua)

  # キーワードの取得
  tag_ul = soup_achievement.article.ul
  # キーワード設定の有無を確認
  keyword_achievement_list = []
  if not tag_ul is None:
    i_a_list = tag_ul.find_all("a")
    for j_a in i_a_list:
      j_title = j_a.get('title')
      keyword_achievement_list.append(j_title)

  """
  施工情報については"dl"タグ内の、"dd"タグにタイトル、"dt"に施工データ が書かれているが、
  全ての閉じタグが最後にまとめて記述されているため、BeautifulSoupのメソッドでは対処できない
  そこで、文字列に変換してから区切ることで該当情報を抽出する
  """
  # 施工情報の取得
  tag_dl = soup_achievement.article.find("dl", {"class": "list-b"})
  # 施工情報設定の有無を確認
  infoKey_achPage_list = []
  infoValue_achPage_list = []
  if not tag_dl is None:
    # タイトルの取得
    i_list_dt = tag_dl.find_all("dt")
    i_title_list = []
    for j_list_dt in i_list_dt:
      # 文字列化し、先頭の不要な文字列(改行、空白等)を削除
      j_list_dt_text = j_list_dt.text.lstrip()
      # 区切り文字("：")の位置を探し、それ以前の文字列を取得する
      j_dt_delimiter = j_list_dt_text.find("：")
      j_title = j_list_dt_text[: j_dt_delimiter]
      i_title_list.append(j_title)

    # 施工データのリスト取得
    i_list_dd = tag_dl.find_all("dd")
    i_data_list = []
    for j_list_dd in i_list_dd:
      # 文字列化し、先頭の不要な文字列(改行、空白等)を削除
      j_list_dd_text = j_list_dd.text.lstrip()
      # 区切り文字("\n")の位置を探し、それ以前の文字列を取得する
      j_dd_delimiter = j_list_dd_text.find("\n")
      j_data = j_list_dd_text[: j_dd_delimiter]
      # 末尾の不要な文字列を削除する
      j_data = j_data.rstrip()
      i_data_list.append(j_data)
    sys.exit()


SystemExit: ignored

## テスト

In [61]:
t0 = tag_dl.find_all("dd")[2].text
t1 = t0.lstrip()
t2 = t1.find("\n")
t3 = t1[:t2]

print(t3.rstrip())

九州三菱自動車販売株式会社様


In [75]:
print(i_list_dd)

[<dd>
                            2021年
                        <dt>
                            所在地：
                        <dd>
                            福岡県                        <dt>
                            クライアント：
                        <dd>
                            九州三菱自動車販売株式会社様                                                <dt>
                            業務内容：
                        <dd>
                            デザイン・設計、サイン・グラフィックデザイン、デザイン監修、内装監理、環境演出装置設計・制作、制作・内装施工                                                                        </dd></dt></dd></dt></dd></dt></dd>, <dd>
                            福岡県                        <dt>
                            クライアント：
                        <dd>
                            九州三菱自動車販売株式会社様                                                <dt>
                            業務内容：
                        <dd>
                            デザイン・設計、サイン・グラフィックデザイン、デザイン監修、内装監理、環境演出装置設計・制作、制作・内装施工                         

In [57]:
pprint.pprint(i_data_list)

['2021年',
 '福岡県                        ',
 '九州三菱自動車販売株式会社様                                                ',
 'デザイン・設計、サイン・グラフィックデザイン、デザイン監修、内装監理、環境演出装置設計・制作、制作・内装施']
